5단계 까지 도출한 사항을 정리하자면,

Weight Capacity (kg)는 Price를 예측할 수 있는 Key feature 이다. Weight Capacity (kg)는 범주형 성격을 보이는데, 

이는 

In [1]:
import os, sys

import pandas as pd
import polars as pl

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import scipy

import sklearn
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import dproc, sgml, sgnn, sgpp, sgutil, custpp

print(sys.version)
for i in [pd, pl, mpl, sns, np, scipy, sklearn, lgb, xgb, cb]:
    try:
        print(i.__name__, i.__version__)
    except:
        pass

2025-02-10 01:23:06.474672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739150586.606831    5272 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739150586.642442    5272 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 01:23:06.955969: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


3.12.6 (main, Sep 30 2024, 02:19:13) [GCC 9.4.0]
pandas 2.2.3
polars 1.12.0
matplotlib 3.8.4
seaborn 0.13.2
numpy 1.26.4
scipy 1.12.0
sklearn 1.5.2
xgboost 2.1.2
catboost 1.2.5


In [2]:
from itertools import combinations

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, ShuffleSplit, train_test_split
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [3]:
sc = sgutil.SGCache('img', 'result')
ss = ShuffleSplit(n_splits = 1, train_size = 0.8, random_state = 123)
kf = KFold(5, random_state = 123, shuffle=True)

files = {
    'train': os.path.join('data', 'train.csv'),
    'train_extra': os.path.join('data', 'training_extra.csv'),
    'test': os.path.join('data', 'test.csv'),
}

t = sc.cache_result(
    'pipeline_2',
    lambda : make_pipeline(
        sgpp.PolarsProcessor(), 
        sgpp.ExprProcessor({
            'Compartments_c' : pl.col('Compartments').cast(pl.String).cast(pl.Categorical)
        }),
        sgpp.PandasCoverter(index_col = 'id'),
        sgpp.ApplyWrapper(
            sgpp.CatArrangerFreq(1, na_value = 'Unknown'),
            ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
        ), 
        custpp.WeightCapacityProcessor()
    ).fit(files['train']),
    rerun = 1
)
df_train = pd.concat(
    [t.transform(files['train']), t.transform(files['train_extra'])], axis = 0
)
df_test = t.transform(files['test'])

target = 'Price'
X_cat = ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color', 'Compartments_c']
X_num = ['Weight Capacity (kg)']

In [8]:
def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict(df[X]), index = df.index).clip(15, 150),
    'score_func': lambda df, prds: root_mean_squared_error(df[target].sort_index(), prds.sort_index()),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'y': target,
}

cb_adapter = sgml.CBAdapter(cb.CatBoostRegressor)
lr_adapter = sgml.SklearnAdapter(LinearRegression)
lgb_adapter = sgml.LGBMAdapter(lgb.LGBMRegressor)
xgb_adapter = sgml.XGBAdapter(xgb.XGBRegressor)

In [19]:
def eval_model(hparams, adapter, case = 0, sp = ss, **args):
    results = list()
    for train_idx, valid_idx in sp.split(df_train, df_train[target]):
        df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
        bidx = df_valid['Weight Capacity (kg)'].isin(df_cv_train['Weight Capacity (kg)'].unique()) & df_valid['Weight Capacity (kg)'].notna() &\
                df_valid['Weight Capacity (kg)'].between(5, 30, inclusive = 'neither')
        if case == 1:
            bidx = ~bidx
        df_valid = df_valid.loc[bidx]
        reg = sgml.train(df_cv_train, hparams, config, adapter, **args)
        results.append(
            (reg, root_mean_squared_error(df_valid[target], make_pipeline(reg[0]['preprocessor'], reg[0]['model']).predict(df_valid[reg[1]])))
        )
    return results

In [38]:
results = eval_model({
    'model_params' : {'n_estimators': 1500, 'learning_rate': 0.1},
    'X_num': X_num, 'X_cat': X_cat,
    'validation_fraction': 0.1
}, cb_adapter, case = 1)

Round:   0%|          | 0/1500 [00:00<?, ?it/s]

In [41]:
sgml.cb_learning_result(results[0][0][0])['valid_result'].idxmin(), results[0][1]

(metric  set         
 RMSE    learn           1499
         validation_0    1499
         validation_1    1493
 dtype: int64,
 38.79740725539745)

In [42]:
results = eval_model({
    'model_params' : {'n_estimators': 500, 'learning_rate': 0.1},
    'X_num': X_num, 'X_cat': X_cat,
    'validation_fraction': 0.1
}, cb_adapter, task_type = 'GPU')

In [44]:
results[0][0]

({'variables': array(['cat__Brand', 'cat__Material', 'cat__Size',
         'cat__Laptop Compartment', 'cat__Waterproof', 'cat__Style',
         'cat__Color', 'cat__Compartments_c', 'pt__Weight Capacity (kg)'],
        dtype=object),
  'valid_shape': (319546, 9),
  'train_shape': (2875908, 9),
  'target': 'Price',
  'model': <catboost.core.CatBoostRegressor at 0x7feb015edeb0>,
  'preprocessor': ColumnTransformer(transformers=[('cat', 'passthrough',
                                   ['Brand', 'Material', 'Size',
                                    'Laptop Compartment', 'Waterproof', 'Style',
                                    'Color', 'Compartments_c']),
                                  ('pt', 'passthrough',
                                   ['Weight Capacity (kg)'])])},
 ['Brand',
  'Laptop Compartment',
  'Compartments_c',
  'Style',
  'Waterproof',
  'Color',
  'Weight Capacity (kg)',
  'Size',
  'Material'])

In [ ]:
def eval_model(hparams, adapter, case = 0, **args):
    results = list()
    for train_idx, valid_idx in kf.split(df_train, df_train[target]):
        df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
        bidx = df_valid['Weight Capacity (kg)'].isin(df_cv_train['Weight Capacity (kg)'].unique()) & df_valid['Weight Capacity (kg)'].notna() &\
                df_valid['Weight Capacity (kg)'].between(5, 30, inclusive = 'neither')
        df_valid, df_valid2 = df_valid.loc[bidx], df_valid.loc[~bidx]
        reg1 = sgml.train(
            df_cv_train, {
            'model_params' : {'n_estimators': 500, 'learning_rate': 0.1},
            'X_num': X_num, 'X_cat': X_cat,
            #'validation_fraction': 0.1
        }, config, cb_adapter, task_type = 'GPU')
        reg2 = sgml.train(
            df_cv_train, {
            'model_params' : {'n_estimators': 500, 'learning_rate': 0.1},
            'X_cat': X_cat, 'X_num': X_num + ['tgt'],
            # 'validation_fraction': 0.1
        }, config, cb_adapter, task_type = 'GPU')
        
        s_merge = pd.concat([
            pd.Series(
                make_pipeline(reg_cb2[0]['preprocessor'], reg_cb2[0]['model']).predict(df_valid1[reg_cb2[1]]), index = df_valid1.index
            ),
            pd.Series(
                make_pipeline(reg_cb[0]['preprocessor'], reg_cb[0]['model']).predict(df_valid2[reg_cb[1]]), index = df_valid2.index
            )
        ], axis = 0)